In [8]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import statistics
import numpy as np
import warnings
import sqlite3
from scipy.stats import norm
import math
import cv2
import os
import json
import time
import sys
from numpy.lib.function_base import append
import xlsxwriter
import matplotlib.pyplot as plt
from os.path import join
from mpl_toolkits.mplot3d import Axes3D
from _collections import deque
from scipy.signal import argrelextrema
import glob

In [9]:
##Computing least-squares solution to equation Ax = b
def plane_fitting_max_point(x,y,z):
    y = y
    x = x
    z = z
    num_points = len(x)
    z_step_size = 1.875

#     ax = plt.subplot(111, projection='3d')
#     ax.scatter(x, y, z, color='b')

    tmp_A = []
    tmp_b = []
    for i in range(len(x)):
        tmp_A.append([x[i], y[i], 1])
        tmp_b.append(z[i])

    b = np.matrix(tmp_b).T
    A = np.matrix(tmp_A)

    from scipy.linalg import lstsq
    fit, residual, rnk, s = lstsq(A, b)
    errors =  A * fit - b
    plane_z = A * fit
    
    index_min_plane_z = np.argmin(plane_z)
    index_max_plane_z = np.argmax(plane_z)

    x_z_min_in_microns = x[index_min_plane_z]
    x_z_max_in_microns = x[index_max_plane_z]
    y_z_min_in_microns = y[index_min_plane_z]
    y_z_max_in_microns = y[index_max_plane_z]

    max_plane_z_diff  = max(plane_z) - min(plane_z)
    
    slope = (max(plane_z) - min(plane_z))/(math.dist([x_z_max_in_microns, y_z_max_in_microns], [x_z_min_in_microns, y_z_min_in_microns]))
#     print("Slope: ", slope)
#     print("solution: %f x + %f y + %f = z" % (fit[0], fit[1], fit[2]))
    return(slope)


In [3]:
conn = sqlite3.connect('/home/adminspin/Desktop/Planarity 2/H01EBB43P-645/H01EBB43P-645.db')
c = conn.cursor()
query = "select aoi_name, x_pos, y_pos, row_index, column_index, best_z, z_difference, stack_size, best_index, focus_metric from focus_sampling_info"
c.execute(query)
aoi_info = c.fetchall()
rej = pd.DataFrame(aoi_info)
rej.rename(columns = {0:'aoi_name',1:'x_pos',2:'y_pos',3:'row_index',4:'column_index',5:'best_z',6:'z_difference',
                    7:'stack_size',8:'best_index',9:'focus_metric'}, inplace = True)
rej

,aoi_name,x_pos,y_pos,row_index,column_index,best_z,z_difference,stack_size,best_index,focus_metric
0,aoi5823,16762.50,39268.75,58,22,8589.375,8590.375,35,0,25.252554
1,aoi5813,11575.00,39268.75,58,12,-1.000,8590.375,21,-1,7.447018
2,aoi5818,14168.75,39268.75,58,17,-1.000,8590.375,21,-1,4.997743


In [4]:
rej = rej[rej['best_z'] > 0]
x = rej['x_pos'].tolist()
y = rej['y_pos'].tolist()
z = rej['best_z'].tolist()
aa = plane_fitting_max_point(x,y,z).tolist()[0]
aa

/tmp/ipykernel_7805/2626816086.py:36: RuntimeWarning: invalid value encountered in true_divide
  slope = (max(plane_z) - min(plane_z))/(math.dist([x_z_max_in_microns, y_z_max_in_microns], [x_z_min_in_microns, y_z_min_in_microns]))


[nan]

In [10]:
df = pd.DataFrame(columns = ['slide_name', 'best_row_slope', 'scan_status'])
for filename in sorted(glob.glob("/home/adminspin/Desktop/Planarity/*/*.db",recursive=True)):
    file = filename
    lst = []
#     file = '/home/adminspin/Desktop/Planarity/example_planarity/JR-20-1000-A4-22/JR-20-1000-A4-22_H01JBA21P-37065/JR-20-1000-A4-22_H01JBA21P-37065.db'
    name = file.split('/')[-1].split('.')[0]
    lst.append(name)
    conn = sqlite3.connect(file)
    c = conn.cursor()
    query = "select aoi_name, x_pos, y_pos, row_index, column_index, best_z, z_difference, stack_size, best_index, focus_metric from focus_sampling_info where grid_id = 1"
    c.execute(query)
    aoi_info = c.fetchall()
    acc = pd.DataFrame(aoi_info)
    acc.rename(columns = {0:'aoi_name',1:'x_pos',2:'y_pos',3:'row_index',4:'column_index',5:'best_z',6:'z_difference',
                        7:'stack_size',8:'best_index',9:'focus_metric'}, inplace = True)
    acc
    acc = acc[acc['best_z'] > 0]
    acc
    x = acc['x_pos'].tolist()
    y = acc['y_pos'].tolist()
    z = acc['best_z'].tolist()
    slope = plane_fitting_max_point(x,y,z).tolist()[0][0]
    lst.append(slope)

    query1 = "select * from acquisition_blob_info"
    c.execute(query1)
    aoi_info1 = c.fetchall()
    test = pd.DataFrame(aoi_info1)
    if len(test)>1:
        lst += ['scanned']
    else:
        lst += ['rejected']

    df.loc[len(df)] = lst
df

fig = px.scatter(df, x = df['slide_name'], y = df['best_row_slope'], color = df['scan_status'], height = 700)
fig.show()

/tmp/ipykernel_7805/2626816086.py:36: RuntimeWarning:

invalid value encountered in true_divide



In [6]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x = df['slide_name'], y = df['best_row_slope'], mode = 'markers', 
#                          marker_color = df['scan_status']))
fig = px.scatter(df, x = df['slide_name'], y = df['best_row_slope'], color = df['scan_status'])
fig.show()